In [1]:
import logging
import os
import json
import base64
from io import BytesIO
from PIL import Image
import subprocess
import time
from collections import namedtuple
from pathlib import Path

import google.cloud.aiplatform as aiplatform
from google.cloud import storage

logging.getLogger().setLevel(logging.INFO)

In [2]:
APP_NAME = 'ViT-model'
MODEL_PT_FILEPATH = 'saved_models/VisionTransformers'
MAR_MODEL_OUT_PATH = 'serve'
handler = 'predictor/handler.py'
MODEL_DISPLAY_NAME = 'ViT-model'
model_version = 1
PROJECT_ID = 'alberto-playground'
BUCKET_NAME = 'alberto-vit-playground'
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_vit"

In [3]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [4]:
# # create directory to save model archive file
# model_output_root = MODEL_PT_FILEPATH
# mar_output_root = MAR_MODEL_OUT_PATH
# additiona_files_base_dir = 'src/model'
# export_path = f"{mar_output_root}/model-store"
# try:
#     Path(export_path).mkdir(parents=True, exist_ok=True)
# except Exception as e:
#     logging.warning(e)
#     # retry after pause
#     time.sleep(2)
#     Path(export_path).mkdir(parents=True, exist_ok=True)
#
# # parse and configure paths for model archive config
# handler_path = (
#     handler.replace("gs://", "/gcs/") + "predictor/handler.py"
#     if handler.startswith("gs://")
#     else handler
# )
# model_artifacts_dir = model_output_root
# extra_files = [
#     os.path.join(additiona_files_base_dir, f)
#     for f in os.listdir(additiona_files_base_dir)]
#
# # define model archive config
# mar_config = {
#     "MODEL_NAME": MODEL_DISPLAY_NAME,
#     "HANDLER": handler_path,
#     "SERIALIZED_FILE": f'{model_artifacts_dir}/ViT.pt',
#     "VERSION": model_version,
#     "EXTRA_FILES": ",".join(extra_files),
#     "EXPORT_PATH": export_path,
# }
#
# # generate model archive command
# archiver_cmd = (
#     "torch-model-archiver --force "
#     f"--model-name {mar_config['MODEL_NAME']} "
#     f"--serialized-file {mar_config['SERIALIZED_FILE']} "
#     f"--handler {mar_config['HANDLER']} "
#     f"--version {mar_config['VERSION']}"
# )
# if "EXPORT_PATH" in mar_config:
#     archiver_cmd += f" --export-path {mar_config['EXPORT_PATH']}"
# if "EXTRA_FILES" in mar_config:
#     archiver_cmd += f" --extra-files {mar_config['EXTRA_FILES']}"
# if "REQUIREMENTS_FILE" in mar_config:
#     archiver_cmd += f" --requirements-file {mar_config['REQUIREMENTS_FILE']}"
#
# # run archiver command
# logging.warning("Running archiver command: %s", archiver_cmd)
# with subprocess.Popen(
#         archiver_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
# ) as p:
#     _, err = p.communicate()
#     if err:
#         raise ValueError(err)


In [5]:
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob(f'{MAR_MODEL_OUT_PATH}/ViT-model.mar')
blob.upload_from_filename('serve/model-store/ViT-model.mar')

In [6]:
! docker build -f predictor/Dockerfile -t $CUSTOM_PREDICTOR_IMAGE_URI ./

Sending build context to Docker daemon  394.6MB
Step 1/24 : FROM pytorch/torchserve:latest-cpu
 ---> 68a3fcae81af
Step 2/24 : USER root
 ---> Using cache
 ---> 74b7dbf2b479
Step 3/24 : RUN apt-get update &&     apt-get install -y software-properties-common &&     add-apt-repository -y ppa:ubuntu-toolchain-r/test &&     apt-get update &&     apt-get install -y gcc-9 g++-9 apt-transport-https ca-certificates gnupg curl
 ---> Using cache
 ---> 6e360930db3d
Step 4/24 : RUN echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" |     tee -a /etc/apt/sources.list.d/google-cloud-sdk.list &&     curl https://packages.cloud.google.com/apt/doc/apt-key.gpg |     apt-key --keyring /usr/share/keyrings/cloud.google.gpg add - &&     apt-get update -y &&     apt-get install google-cloud-sdk -y
 ---> Using cache
 ---> bfe0359200e4
Step 5/24 : USER model-server
 ---> Using cache
 ---> 2d2bde191019
Step 6/24 : RUN python3 -m pip install --upgrade p

In [7]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/alberto-playground/pytorch_predict_vit]

d321a13c: Preparing 
6e639786: Preparing 
ca3ba6bc: Preparing 
b3e72f47: Preparing 
e1b71f7f: Preparing 
7b334d17: Preparing 
001bafce: Preparing 
bf18a086: Preparing 
7cf25f52: Preparing 
fa8107fa: Preparing 
24bd1a34: Preparing 
0b544b4c: Preparing 
613e1d99: Preparing 
b3c8b2c4: Preparing 
0ae33361: Preparing 
a3ba6bc: Pushed   11.69MB/11.68MBlatest: digest: sha256:583ce1a37e1819f95d2ff0351eccd9ed65dc44813938114456b6058eb70706e2 size: 3672


In [8]:
model_display_name = f"{APP_NAME}-v{model_version}"
model_description = "PyTorch Image classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [9]:

model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
    artifact_uri=f'gs://{BUCKET_NAME}/{MAR_MODEL_OUT_PATH}',
)

model.wait()

print(model.display_name)
print(model.resource_name)

Creating Model
Create Model backing LRO: projects/634066980332/locations/us-central1/models/7447399074229125120/operations/9194591976400355328
Model created. Resource name: projects/634066980332/locations/us-central1/models/7447399074229125120@1
To use this Model in another session:
model = aiplatform.Model('projects/634066980332/locations/us-central1/models/7447399074229125120@1')
ViT-model-v1
projects/634066980332/locations/us-central1/models/7447399074229125120


In [10]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

Creating Endpoint
Create Endpoint backing LRO: projects/634066980332/locations/us-central1/endpoints/3103257220188471296/operations/6071345634818916352
Endpoint created. Resource name: projects/634066980332/locations/us-central1/endpoints/3103257220188471296
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/634066980332/locations/us-central1/endpoints/3103257220188471296')


In [11]:
traffic_percentage = 100
machine_type = "n1-standard-4"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync,
)

Deploying model to Endpoint : projects/634066980332/locations/us-central1/endpoints/3103257220188471296
Deploy Endpoint model backing LRO: projects/634066980332/locations/us-central1/endpoints/3103257220188471296/operations/8654160021115895808
Endpoint model deployed. Resource name: projects/634066980332/locations/us-central1/endpoints/3103257220188471296


resource name: projects/634066980332/locations/us-central1/endpoints/3103257220188471296

In [12]:
endpoint_display_name = f"{APP_NAME}-endpoint"
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

Endpoint display name = ViT-model-endpoint resource id =projects/634066980332/locations/us-central1/endpoints/3103257220188471296 


In [13]:
endpoint.list_models()

[id: "5802731989215739904"
 model: "projects/634066980332/locations/us-central1/models/7447399074229125120"
 display_name: "ViT-model-v1"
 create_time {
   seconds: 1690692433
   nanos: 646750000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

In [14]:
def limit_img_size(img, target_filesize, tolerance=5):
    # img = img_orig = Image.open(img_filename)
    aspect = img.size[0] / img.size[1]

    while True:
        with BytesIO() as buffer:
            img.save(buffer, format="JPEG")
            data = buffer.getvalue()
        filesize = len(data)    
        size_deviation = filesize / target_filesize
        print("size: {}; factor: {:.3f}".format(filesize, size_deviation))

        if size_deviation <= (100 + tolerance) / 100:
            # filesize fits
            return data
        else:
            # filesize not good enough => adapt width and height
            # use sqrt of deviation since applied both in width and height
            new_width = img.size[0] / size_deviation**0.5    
            new_height = new_width / aspect
            # resize from img_orig to not lose quality
            img = img.resize((int(new_width), int(new_height)))

In [15]:
blob = bucket.get_blob('samples/pexels-helena-lopes-1996332.jpg')    
img = Image.open(BytesIO(blob.download_as_bytes()))
image_bytes = limit_img_size(img,  15000,tolerance = 5)
encoded_string = base64.b64encode(image_bytes)

size: 281851; factor: 18.790
size: 17194; factor: 1.146
size: 14442; factor: 0.963


In [16]:
test_instance = [
    {
    "file": {
        "filename": "pexels-helena-lopes-1996332.jpg",
        "content": f"{str(encoded_string.decode('utf-8'))}" }
    }
]

In [17]:
prediction = endpoint.predict(instances=test_instance)

In [18]:
prediction

Prediction(predictions=[{'response': 'dog', 'probabilities': {'cat': 0.06561080366373062, 'ship': 0.1539709866046906, 'dog': 0.1819493472576141, 'frog': 0.1282880455255508, 'deer': 0.06153243407607079, 'automobile': 0.07853943854570389, 'bird': 0.06922928243875504, 'horse': 0.09804531931877136, 'airplane': 0.1399828493595123, 'truck': 0.02285150066018105}}], deployed_model_id='5802731989215739904', model_version_id='1', model_resource_name='projects/634066980332/locations/us-central1/models/7447399074229125120', explanations=None)

In [19]:
test_instance = [{'link': 'gs://alberto-vit-playground/samples/pexels-helena-lopes-1996332.jpg'}]

In [20]:
prediction = endpoint.predict(instances=test_instance)

In [21]:
prediction

Prediction(predictions=[{'response': 'dog', 'probabilities': {'cat': 0.06622505933046341, 'dog': 0.18122299015522, 'airplane': 0.1400623470544815, 'truck': 0.02236789651215076, 'frog': 0.1266641616821289, 'deer': 0.06160113960504532, 'ship': 0.1543110311031342, 'horse': 0.0966719463467598, 'bird': 0.07047797739505768, 'automobile': 0.08039550483226776}}], deployed_model_id='5802731989215739904', model_version_id='1', model_resource_name='projects/634066980332/locations/us-central1/models/7447399074229125120', explanations=None)